In [251]:
import scipy as sc
import numpy as np
#!pip install gpxpy
import gpxpy
from geopy import distance
import json
import pandas as pd
import math
!pip install fit2gpx
from fit2gpx import Converter

  Using cached fit2gpx-0.0.6.tar.gz (8.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached fitdecode-0.10.0-py3-none-any.whl (90 kB)
  Created wheel for fit2gpx: filename=fit2gpx-0.0.6-py3-none-any.whl size=8194 sha256=8e328d85f819b8100d1510b484851ee96025f27f75e523cfdd150be127c96aa7
  Stored in directory: /Users/wjones/Library/Caches/pip/wheels/fb/0f/68/223e3b1e6827c306b647aa5c72454a2388f296cff8c553d067
Successfully built fit2gpx


# Putting it all together:

1. read gpx file(s) into an array with elevations and coordinates
2. read segement data into array with elevations and coordinates
3. get total distance and total elevation difference from segment
4. sweep through gpx arrays to find sections that roughly fit the segment
5. calculate elapsed times for those sections
6. use DTW to see if segments are actually similar
    - if they are, compare times.

- Thought - skip the prelim data sweep and just use DTW

In [215]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            #print(cost)
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

In [216]:
def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

In [217]:
from geopy import distance
import math
# method to calculate distances between point
def distanceBetweenPoints(p1, p2):
    flat_distance = distance.distance(p1[:2], p2[:2]).m
    # print(flat_distance)
    euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
    return euclidian_distance

In [218]:
from geopy import Point
import math
# method to make GPX file into array of elevations, times, distances
def file_to_elevation_arr(fileName):
        gpx_file = open(fileName, 'r')
        gpx = gpxpy.parse(gpx_file)
        elevevation_time_and_distance = []
        pointOne = None
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    pointTwo = point
                    distance = 0
                    if pointOne:
                        p1 = Point([pointOne.latitude, pointOne.longitude, pointOne.elevation])
                        p2 = Point([pointTwo.latitude, pointTwo.longitude, pointTwo.elevation])
                        distance = distanceBetweenPoints(p1, p2)
                    pointOne = point
                    #print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))
                    elevevation_time_and_distance.append((point.elevation,point.time, distance))
        return elevevation_time_and_distance

In [219]:
fileOne = "data/Walt's Strava/activities/5378707142.gpx"
fileTwo  = "data/Walt's Strava/activities/4812864781.gpx"

In [220]:
arrOne = file_to_elevation_arr(fileOne)
#print(arrOne)
arrTwo = file_to_elevation_arr(fileTwo)

In [221]:
segment = "data/Elevation JSON/segment_location_data_elevation_0.json"

In [222]:
df = pd.read_json(segment)
df.head()


,lat,lon,elev_meters,distance
0,34.09464,-117.71377,355.14,0.000000
1,34.09467,-117.71362,355.22,14.236931
2,34.09467,-117.71327,355.12,32.298915
3,34.09476,-117.71181,356.33,135.106672
4,34.09474,-117.71131,356.34,46.194346


In [223]:
def dataframe_segment_to_list(df):
    sg =[]
    # Iterate over each row
    for index, rows in df.iterrows():
        # Create list for the current row
        my_list =[rows[2], rows[3]]
        #print(my_list)
        
        # append the list to the final list
        sg.append(my_list)
    return sg
  

In [224]:
#print(segment_data)
def total_segment_elevation(segment_data):
    total_elev = segment_data[len(segment_data)-1][0] - segment_data[0][0]
    return total_elev

def total_segment_distance(segment_data):
    total_distance = 0
    for i in segment_data:
        total_distance += i[1].item()
    return total_distance

In [237]:
def sweep_user_activity_old(user_data, segement_data, x):
    list_of_matches = [] # list of lists

    seg_distance = total_segment_distance(segement_data)

    #print(seg_distance)
    seg_elevation = total_segment_elevation(segement_data)
    #print(seg_elevation)
    
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    first_window = 0
    while user_distance < seg_distance and first_window < len(user_data):
        # until we get to the window 
        user_elev = user_data[first_window][0]-user_start_elev
        user_distance += user_data[first_window][2]
        first_window += 1
    delta_distance = user_distance - user_data[0][2]
    # now we have the window, so we need to iterate through and see if the profiles match
    second_window = 0
    second_distance = 0
    user_elev_two = user_data[0][0]
    while user_distance < user_data[-1][2] and first_window < len(user_data):
        user_elev_two = user_data[first_window][0]
        user_elev = user_data[second_window][0]
        if user_elev_two - user_elev > seg_elevation-5 and  user_elev_two - user_elev < seg_elevation+5:
            list_of_matches.append(user_data[second_window:first_window])
            #print(first_window, second_window)
            # add match to list
        user_distance += user_data[first_window][2]
        second_distance += user_data[second_window][2]
        first_window +=1
        second_window +=1
    return list_of_matches




In [238]:
# What does this need to do?
# Go to segment distance, when the segment distance matches the user window, compare elevations
def sweep_user_activity(user_data, segement_data, window):
    list_of_matches = [] # list of lists
    seg_distance = total_segment_distance(segement_data) # total segment distance
    seg_elevation = total_segment_elevation(segement_data) # total segment elevation
    
    # go to the correct distance in user
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    first_window = 0
    while user_distance < seg_distance and first_window < len(user_data):
        user_elev = user_data[first_window][0]-user_start_elev
        user_distance += user_data[first_window][2]
        first_window+=1
    second_window = 0 
    second_distance = 0
    while first_window < len(user_data)-1: 
        while user_distance-second_distance < seg_distance and first_window < len(user_data)-1:
            user_distance += user_data[first_window][2]
            first_window+=1
        user_elev_two = user_data[first_window][0]
        user_elev = user_data[second_window][0]
        second_distance += user_data[second_window][2]
        if user_elev_two - user_elev > seg_elevation-window and  user_elev_two - user_elev < seg_elevation+window:
            list_of_matches.append(user_data[second_window:first_window])
        second_window+= 1
    return list_of_matches
        


In [227]:
# TODO: add DTW algorithm match

In [239]:
claremont_turkey_trot = dataframe_segment_to_list(df)
#print(claremont_turkey_trot)
arr = sweep_user_activity(arrTwo, claremont_turkey_trot, 5)
#print(arr)

In [229]:
#print(arr[0])

In [230]:
import matplotlib.pyplot as plt
    

In [231]:
import numpy as np
import matplotlib.pyplot as plt
def graphing(user, segment):
    total_user_distance = 0
    total_segment_distance1 = 0
    user_start_elev = user[0][0]
    segment_start_elev = segment[0][0]
    user_dtw_list = []
    segment_dtw_list = []
    for i in range(len(user)):
        total_user_distance += user[i][2]
        user_dtw_list.append(user[i][0]-user_start_elev)
        #plt.scatter(total_user_distance, user[i][0])
        #print(arr[0][i][0])
    for j in range (len(segment)):
         total_segment_distance1 += segment[j][1]
         segment_dtw_list.append(segment[j][0]-segment_start_elev)
         #plt.scatter(total_segment_distance1, segment[j][0])
    #plt.draw()
    return (user_dtw_list, segment_dtw_list)

In [247]:
l = graphing(arr[0], claremont_turkey_trot)
def DTW_comparison(user, segment):
    distances = []
    user_times = []
    d_t = []
    for pot in range(len(user)):
        t1 = user[pot][0][1]
        t2 = user[pot][-1][1]
        #print(t1)
        user_times.append((t2-t1).total_seconds())
        l = graphing(user[pot], segment)
        tempDTW = dtw(l[0], l[1])
        distances.append(tempDTW[-1][-1])
    for d in range(len(distances)):
        d_t.append((distances[d],user_times[d]))
    return d_t
    


In [248]:
d = DTW_comparison(arr, claremont_turkey_trot)


In [249]:
print(d)    

[(20697.829999999834, 1682.0), (20697.829999999834, 1681.0), (20035.899999999943, 1629.0), (20035.899999999943, 1628.0), (20035.899999999943, 1627.0)]


In [272]:
conv = Converter() 
def fit_to_gpx(fileDirectory):
    conv.fit_to_gpx_bulk(dir_in=fileDirectory, dir_out='data/User')



In [273]:
#fit_to_gpx('data/james_activities')

/Users/wjones/opt/anaconda3/envs/dsa/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 194); adding dummy dev data...
  warnings.warn(msg)
/Users/wjones/opt/anaconda3/envs/dsa/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 236); adding dummy dev data...
  warnings.warn(msg)
/Users/wjones/opt/anaconda3/envs/dsa/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 287); adding dummy dev data...
  warnings.warn(msg)
/Users/wjones/opt/anaconda3/envs/dsa/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; ch

In [253]:
def compare_user_activities_to_segment(segment):
    pass

In [267]:
!pip install dtaidistance
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis

In [260]:
l = graphing(arr[1], claremont_turkey_trot)
print(len(l[0]))
print(len(l[1]))
path = dtw.warping_path(l[0], l[1])
dtwvis.plot_warping(l[0], l[1], path, filename="warp.png")

1596
125


(None, None)

In [268]:
d = dtw.distance_fast(l[0], l[1], use_pruning=True)
print(d)

The compiled dtaidistance C library is not available.
See the documentation for alternative installation options.


Exception: The compiled dtaidistance C library is not available.
See the documentation for alternative installation options.

In [269]:
dtw.try_import_c(verbose=True)

Cannot import OMP-based library (dtw_cc_omp)
Cannot import Numpy-based library (dtw_cc_numpy)

Not all libraries are available in your installation. 
You can rerun the compilation from source or pip install in verbose mode:
pip install -vvv --upgrade --force-reinstall --no-deps --no-binary dtaidistance dtaidistance
In case you need to use an older version of numpy, compile against your current installation:
pip install -vvv --upgrade --force-reinstall --no-deps --no-build-isolation --no-binary dtaidistance dtaidistance

Share the following information when submitting a bug report:
== Packages ==
- Cannot import OMP-based library (dtw_cc_omp)
- dlopen(/Users/wjones/opt/anaconda3/envs/dsa/lib/python3.9/site-packages/dtaidistance/dtw_cc_omp.cpython-39-darwin.so, 0x0002): Library not loaded: /usr/local/lib/libomp.dylib
  Referenced from: <FD5B81E3-F92A-3645-8432-9E0A64CE90AC> /Users/wjones/opt/anaconda3/envs/dsa/lib/python3.9/site-packages/dtaidistance/dtw_cc_omp.cpython-39-darwin.so
  Rea

False